<a href="https://colab.research.google.com/github/vibhorjoshi/Challenges-/blob/main/CMI_%7C_LGBM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:

# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES
# TO THE CORRECT LOCATION (/kaggle/input) IN YOUR NOTEBOOK,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

import os
import sys
from tempfile import NamedTemporaryFile
from urllib.request import urlopen
from urllib.parse import unquote, urlparse
from urllib.error import HTTPError
from zipfile import ZipFile
import tarfile
import shutil

CHUNK_SIZE = 40960
DATA_SOURCE_MAPPING = 'child-mind-institute-problematic-internet-use:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-competitions-data%2Fkaggle-v2%2F81933%2F9643020%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240924%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240924T113655Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3Dc2d9dcaffb21cc716e2488640dfe3b1f09c4d471ddc69496945a106fb5500abb7e61890e0eeed7300fbf3a3a35b820812ce49ab35b191ef886204137153d7520f9cbe818e59c7f8649229329169f2bf74e916b131fe203772da82f9bf767a5b1c91600c5f2a86f5203bc77d227e50bdefa0498a83a55c0c70627da6feed772bbe62bff27c568672e639f2d3b117a0bca129dc0036bdf6e93c8566e28e2a43bfb00b55edd03667647988f0544a283c1c20d84834b48fb8a6fd7fc301d15f5c1eb3c1b70225bd4d1c0354134d3a81b34490bdcd2998a1307dde0c8cdb3023e72079b88cc9d0e97c5d37d9c7b857d71e463469cda3348344e3ee391fa8d27ad92a6'

KAGGLE_INPUT_PATH='/kaggle/input'
KAGGLE_WORKING_PATH='/kaggle/working'
KAGGLE_SYMLINK='kaggle'

!umount /kaggle/input/ 2> /dev/null
shutil.rmtree('/kaggle/input', ignore_errors=True)
os.makedirs(KAGGLE_INPUT_PATH, 0o777, exist_ok=True)
os.makedirs(KAGGLE_WORKING_PATH, 0o777, exist_ok=True)

try:
  os.symlink(KAGGLE_INPUT_PATH, os.path.join("..", 'input'), target_is_directory=True)
except FileExistsError:
  pass
try:
  os.symlink(KAGGLE_WORKING_PATH, os.path.join("..", 'working'), target_is_directory=True)
except FileExistsError:
  pass

for data_source_mapping in DATA_SOURCE_MAPPING.split(','):
    directory, download_url_encoded = data_source_mapping.split(':')
    download_url = unquote(download_url_encoded)
    filename = urlparse(download_url).path
    destination_path = os.path.join(KAGGLE_INPUT_PATH, directory)
    try:
        with urlopen(download_url) as fileres, NamedTemporaryFile() as tfile:
            total_length = fileres.headers['content-length']
            print(f'Downloading {directory}, {total_length} bytes compressed')
            dl = 0
            data = fileres.read(CHUNK_SIZE)
            while len(data) > 0:
                dl += len(data)
                tfile.write(data)
                done = int(50 * dl / int(total_length))
                sys.stdout.write(f"\r[{'=' * done}{' ' * (50-done)}] {dl} bytes downloaded")
                sys.stdout.flush()
                data = fileres.read(CHUNK_SIZE)
            if filename.endswith('.zip'):
              with ZipFile(tfile) as zfile:
                zfile.extractall(destination_path)
            else:
              with tarfile.open(tfile.name) as tarfile:
                tarfile.extractall(destination_path)
            print(f'\nDownloaded and uncompressed: {directory}')
    except HTTPError as e:
        print(f'Failed to load (likely expired) {download_url} to path {destination_path}')
        continue
    except OSError as e:
        print(f'Failed to load {download_url} to path {destination_path}')
        continue

print('Data source import complete.')


[==================================================] 6667858795 bytes downloaded
Downloaded and uncompressed: child-mind-institute-problematic-internet-use
Data source import complete.


<div style="border: 5px solid rgb(100, 150, 56); border-radius: 20px; width: 70%; height: 300px; background-color: #f0f0f0; display: flex; justify-content: center; align-items: center; position: relative;">
    <img src="https://www.kaggle.com/code/abdmental01/cmi-single-lgbm?scriptVersionId=197933942&cellId=1" alt="CMI Single LGBM" style="border-radius: 30px; width: 100%; height: 100%; position: absolute; top: 0; left: 0; opacity: 0.5;">
    <div style="position: relative; z-index: 1; background-color: rgba(73, 50, 41, 0.5); padding: 10px; border-radius: 10px; color: orange;">
        <h1 style="color: rgb(100, 150, 56); text-align: center;">CMI Single LightGBM Model</h1>
    </div>
</div>



 # CMI Single LightGBM Model

This notebook demonstrates a LightGBM model for the **CMI Single** competition. The model is built using **LightGBM** (LGBM), a popular gradient boosting framework, to handle large datasets efficiently.

## 1. Introduction

The **LightGBM** framework is a fast, distributed, high-performance gradient boosting library based on decision tree algorithms, and it is widely used for solving classification, regression, and ranking problems. This notebook implements a **Single LightGBM Model** to predict the target values.

### Key Steps:
- Import necessary libraries.
- Load and preprocess the dataset.
- Define and train the LightGBM model.
- Evaluate model performance.



## 2. Import Libraries


We start by importing the necessary libraries, including LightGBM, for building and training the model, and other utilities for data manipulation.

```python
import lightgbm as lgb
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

## 3. Data preprocessing


# Load the dataset

data = pd.read_csv('/path/to/dataset.csv')

# Define target and features
X = data.drop(columns=['target'])
y = data['target']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

## 4. Creating LightGBM Dataset

# Create LightGBM dataset
train_data = lgb.Dataset(X_train, label=y_train)
test_data = lgb.Dataset(X_test, label=y_test, reference=train_data)

#5 .Defining Hyperparameters

params = {
    'objective': 'regression',
    'metric': 'rmse',
    'boosting_type': 'gbdt',
    'learning_rate': 0.05,
    'num_leaves': 31,
    'max_depth': -1,
    'feature_fraction': 0.9,
    'bagging_fraction': 0.8,
    'bagging_freq': 5,
    'verbosity': -1,
    'seed': 42
}

# 6. Training the Model

# Train the model
model = lgb.train(
    params,
    train_data,
    valid_sets=[test_data],
    num_boost_round=1000,
    early_stopping_rounds=50,
    verbose_eval=100
)
# 7. Model Evaluation

# Predict on the test set
y_pred = model.predict(X_test, num_iteration=model.best_iteration)

# Calculate RMSE
rmse = mean_squared_error(y_test, y_pred, squared=False)
print(f'RMSE: {rmse}')


## 8. Future steps could include:

Hyperparameter tuning using grid search or Bayesian optimization.
Incorporating additional features or performing feature engineering.






In [3]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/child-mind-institute-problematic-internet-use/sample_submission.csv
/kaggle/input/child-mind-institute-problematic-internet-use/train.csv
/kaggle/input/child-mind-institute-problematic-internet-use/test.csv
/kaggle/input/child-mind-institute-problematic-internet-use/data_dictionary.csv
/kaggle/input/child-mind-institute-problematic-internet-use/series_train.parquet/id=c6ead822/part-0.parquet
/kaggle/input/child-mind-institute-problematic-internet-use/series_train.parquet/id=25a3a723/part-0.parquet
/kaggle/input/child-mind-institute-problematic-internet-use/series_train.parquet/id=5ed5f205/part-0.parquet
/kaggle/input/child-mind-institute-problematic-internet-use/series_train.parquet/id=0c4eabac/part-0.parquet
/kaggle/input/child-mind-institute-problematic-internet-use/series_train.parquet/id=9a2082cf/part-0.parquet
/kaggle/input/child-mind-institute-problematic-internet-use/series_train.parquet/id=f8bf4a1c/part-0.parquet
/kaggle/input/child-mind-institute-problematic-inte

In [4]:
!pip install optuna colorama catboost

In [5]:
import numpy as np
import pandas as pd
import polars as pl
import pandas as pd
from sklearn.base import clone
from copy import deepcopy
import optuna
from scipy.optimize import minimize
import os

import re
from colorama import Fore,Style

from tqdm import tqdm
from IPython.display import clear_output
from concurrent.futures import ThreadPoolExecutor
import warnings
warnings.filterwarnings('ignore')
pd.options.display.max_columns=None

import lightgbm as lgb
from catboost import CatBoostRegressor , CatBoostClassifier
from xgboost import XGBRegressor
from sklearn.ensemble import VotingRegressor
from sklearn.model_selection import *
from sklearn.metrics import *
SEED=42
n_splits=5





# Basic preprocessing

![Basic Preprocess](https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcQ3pWgfTPNSngGrf3M5P1mLtqdYvUT_CypbnQ&s)




In [ ]:
%%time

train=pd.read_parquet('/kaggle/input/child-mind-institute-problematic-internet-use/series_train.parquet')
test=pd.read_csv('/kaggle/input/child-mind-institute-problematic-internet-use/test.csv')
sample=pd.read_csv('/kaggle/input/child-mind-institute-problematic-internet-use/sample_submission.csv')

def process_file(filename,dirname):
    df = pd.read_parquet(os.path.join(dirname, filename, 'part-0.parquet'))
    df.drop('step', axis=1, inplace=True)
    return df.describe().values.reshape(-1), filename.split('=')[1]


def load_time_series(dirname) -> pd.DataFrame:
    ids = os.listdir(dirname)

    with ThreadPoolExecutor() as executor:
        results = list(tqdm(executor.map(lambda fname: process_file(fname, dirname), ids), total=len(ids)))

    stats, indexes = zip(*results)

    df = pd.DataFrame(stats, columns=[f"stat_{i}" for i in range(len(stats[0]))])
    df['id'] = indexes

    return df

train_ts = load_time_series("/kaggle/input/child-mind-institute-problematic-internet-use/series_train.parquet")
test_ts = load_time_series("/kaggle/input/child-mind-institute-problematic-internet-use/series_test.parquet")
time_series_cols = train_ts.columns.tolist()
time_series_cols.remove("id")

train = pd.merge(train, train_ts, how="left", on='id')
test = pd.merge(test, test_ts, how="left", on='id')

train = train.drop('id',axis=1)
test = test.drop('id',axis=1)

featuresCols = ['Basic_Demos-Enroll_Season', 'Basic_Demos-Age', 'Basic_Demos-Sex',
       'CGAS-Season', 'CGAS-CGAS_Score', 'Physical-Season', 'Physical-BMI',
       'Physical-Height', 'Physical-Weight', 'Physical-Waist_Circumference',
       'Physical-Diastolic_BP', 'Physical-HeartRate', 'Physical-Systolic_BP',
       'Fitness_Endurance-Season', 'Fitness_Endurance-Max_Stage',
       'Fitness_Endurance-Time_Mins', 'Fitness_Endurance-Time_Sec',
       'FGC-Season', 'FGC-FGC_CU', 'FGC-FGC_CU_Zone', 'FGC-FGC_GSND',
       'FGC-FGC_GSND_Zone', 'FGC-FGC_GSD', 'FGC-FGC_GSD_Zone', 'FGC-FGC_PU',
       'FGC-FGC_PU_Zone', 'FGC-FGC_SRL', 'FGC-FGC_SRL_Zone', 'FGC-FGC_SRR',
       'FGC-FGC_SRR_Zone', 'FGC-FGC_TL', 'FGC-FGC_TL_Zone', 'BIA-Season',
       'BIA-BIA_Activity_Level_num', 'BIA-BIA_BMC', 'BIA-BIA_BMI',
       'BIA-BIA_BMR', 'BIA-BIA_DEE', 'BIA-BIA_ECW', 'BIA-BIA_FFM',
       'BIA-BIA_FFMI', 'BIA-BIA_FMI', 'BIA-BIA_Fat', 'BIA-BIA_Frame_num',
       'BIA-BIA_ICW', 'BIA-BIA_LDM', 'BIA-BIA_LST', 'BIA-BIA_SMM',
       'BIA-BIA_TBW', 'PAQ_A-Season', 'PAQ_A-PAQ_A_Total', 'PAQ_C-Season',
       'PAQ_C-PAQ_C_Total', 'SDS-Season', 'SDS-SDS_Total_Raw',
       'SDS-SDS_Total_T', 'PreInt_EduHx-Season',
       'PreInt_EduHx-computerinternet_hoursday','sii']

featuresCols += time_series_cols

train = train[featuresCols]
train = train.dropna(subset='sii')

cat_c = ['Basic_Demos-Enroll_Season','CGAS-Season','Physical-Season','Fitness_Endurance-Season','FGC-Season',
 'BIA-Season','PAQ_A-Season','PAQ_C-Season','SDS-Season','PreInt_EduHx-Season']

def update(df):

    global cat_c
    for c in cat_c :
        df[c] = df[c].fillna('Missing')
        df[c] = df[c].astype('category')

    return df

train = update(train)
test = update(test)

def create_mapping(column, dataset):
    unique_values = dataset[column].unique()
    return {value: idx for idx, value in enumerate(unique_values)}

for col in cat_c:

    mapping = create_mapping(col, train)
    mappingTe = create_mapping(col, test)

    train[col] = train[col].replace(mapping).astype(int)
    test[col] = test[col].replace(mappingTe).astype(int)



In [ ]:
%%time

train.head()

![Modeling | Single LGBM](https://lightgbm.readthedocs.io/en/latest/_images/LightGBM_logo_black_text.svg)



In [ ]:
%%time

def quadratic_weighted_kappa(y_true,y_pred):
    return cohen_kappa_score(y_true,y_pred,weights='quadratic')

def threshold_Rounder(oof_non_rounded,thresholds):
    return np.where(oof_non_rounded < thresholds[0],0,np.where(oof_non_rounded < thresholds[2],2,3))

def evaluate_predictions(thresholds , y_true, oof_non_rounded ):
    rounded_p=threshold_Rounder(oof_non_rounded,thresholds)
    return -quadratic_weighted_kappa(y_true, rounded_p)

def TrainML(model_class,test_data):

    x=train.drop(['sii'],axis=1)
    y=train['sii']

    SKF=StraifiedKFold(n_splits=n_splits,shuffle=True)

    train_S=[]
    test_=[]

     oof_non_rounded[test_idx] = y_val_pred
        y_val_pred_rounded = y_val_pred.round(0).astype(int)
        oof_rounded[test_idx] = y_val_pred_rounded

        train_kappa = quadratic_weighted_kappa(y_train, y_train_pred.round(0).astype(int))
        val_kappa = quadratic_weighted_kappa(y_val, y_val_pred_rounded)

        train_S.append(train_kappa)
        test_S.append(val_kappa)

        test_preds[:, fold] = model.predict(test_data)

        print(f"Fold {fold+1} - Train QWK: {train_kappa:.4f}, Validation QWK: {val_kappa:.4f}")
        clear_output(wait=True)

    print(f"Mean Train QWK --> {np.mean(train_S):.4f}")
    print(f"Mean Validation QWK ---> {np.mean(test_S):.4f}")

    KappaOPtimizer = minimize(evaluate_predictions,
                              x0=[0.5, 1.5, 2.5], args=(y, oof_non_rounded),
                              method='Nelder-Mead') # Nelder-Mead | # Powell
    assert KappaOPtimizer.success, "Optimization did not converge."

    oof_tuned = threshold_Rounder(oof_non_rounded, KappaOPtimizer.x)
    tKappa = quadratic_weighted_kappa(y, oof_tuned)

    print(f"----> || Optimized QWK SCORE :: {Fore.CYAN}{Style.BRIGHT} {tKappa:.3f}{Style.RESET_ALL}")

    tpm = test_preds.mean(axis=1)
    tpTuned = threshold_Rounder(tpm, KappaOPtimizer.x)

    submission = pd.DataFrame({
        'id': sample['id'],
        'sii': tpTuned
    })

    return submission









In [ ]:
%%time

Params = {'learning_rate': 0.04603534510792164, 'max_depth': 12, 'num_leaves': 478, 'min_data_in_leaf': 13,
          'feature_fraction': 0.8935304204489449, 'bagging_fraction': 0.7840117449237969, 'bagging_freq': 4,
          'lambda_l1': 6.596560434072009, 'lambda_l2': 2.680080551210706e-06}

Light = lgb.LGBMRegressor(**Params,random_state=SEED, verbose=-1,n_estimators=200)
Submission = TrainML(Light,test)

In [ ]:
%%time

Submission.to_csv('submission.csv', index=False)
print(Submission['sii'].value_counts())